In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
parent_dir_path = os.path.dirname(os.getcwd())
raw_data_path = os.path.join(parent_dir_path,'data'+ os.path.sep +'raw'+os.path.sep)
segment_data_path =  os.path.join(parent_dir_path,'data'+ os.path.sep +'segments'+os.path.sep)
models_path =  os.path.join(parent_dir_path,'models'+os.path.sep+'ppo'+os.path.sep)
logs_path = os.path.join(parent_dir_path,'logs'+os.path.sep+'tensorboard'+os.path.sep)
stats_path = os.path.join(parent_dir_path,'logs'+os.path.sep+'records'+os.path.sep)


In [3]:
crma_details = pd.read_excel(raw_data_path + 'CRMAccountDetails.xlsx')
selected_regions = ['Great Lakes','South','West','Northeast']
selected_programsTypes = ['ADN','PN','BSN','Diploma']
crma_details = crma_details[crma_details['ProgramType'].isin(selected_programsTypes)]
crma_details = crma_details[crma_details['Region'].isin(selected_regions)]
Teas_reg_data = pd.read_excel(raw_data_path + 'TEAS Registrations.xlsx')
payment_df = pd.ExcelFile(raw_data_path + 'First Payments Existing Customers.xlsx')
all_paymnt_df = pd.read_excel(payment_df, 'All Pmt Plans')
multi_pmt_plan_df = pd.read_excel(payment_df, 'Multi Pmt Plans Only')
Teas_test_data = pd.read_excel(raw_data_path +'TEASTestDetails.xlsx')

In [4]:
crma_grp = crma_details.groupby(['InstitutionTypeName','SchoolType','ProgramType','Region'])['InstitutionID'].apply(list).reset_index()
teas_reg_grp = Teas_reg_data.groupby(['Year','Quarter'])['#UsersRegistered'].sum().reset_index()
teas_test_df = Teas_test_data[Teas_test_data['Year'].isin([2016,2017,2018,2019,2020,2021,2022,2023])]
teas_test_df_grp = teas_test_df.groupby(['Year', 'Quarter']).sum().reset_index()

In [5]:
for indx,row in crma_grp.iterrows():
    crma_sel_instLst = row['InstitutionID']
    all_paymnt_df_sel = all_paymnt_df[all_paymnt_df['InstitutionID'].isin(crma_sel_instLst)]    
    all_paymnt_df_sel_grp = all_paymnt_df_sel.groupby(['PmtYear','PmtQuarter']).agg({
                                                         'InstitutionID': lambda x: list(x), 
                                                         'BilledStudentsPmt1':'sum'
                                                        }).reset_index() 
    
    key = row['InstitutionTypeName'] + '_'+  row['SchoolType']+ '_'+  row['ProgramType'] + '_' + row['Region']
    
    if len(all_paymnt_df_sel_grp)==30 : 
        df_bl_ts_reg=all_paymnt_df_sel_grp.merge(teas_reg_grp, left_on=['PmtYear', 'PmtQuarter'], 
                                             right_on=['Year', 'Quarter']).drop(['PmtYear','PmtQuarter','InstitutionID'],axis=1)
        
        df_bl_ts_reg_test=teas_test_df_grp.merge(df_bl_ts_reg,on = ['Year', 'Quarter']) 
        df = df_bl_ts_reg_test.copy()
        df.to_csv(segment_data_path + key + '.csv')